In [2]:
import csv
import pandas as pd
import datetime as dt
from datetime import datetime
from itertools import islice
import numpy as np
from openpyxl import load_workbook
import collections
import openpyxl as op
import math
import xlsxwriter

In [3]:
r = 0.147642304763359
alpha = 0.351575403514246
a = 0.327857205096383
b = 1.36694414288691

In [4]:
exportFile = 'Model construction CWM bgnbd2.xlsx'
writer = pd.ExcelWriter(exportFile, engine = 'openpyxl')
grgFile = "GRG Running.xlsx"
writer2 = pd.ExcelWriter(grgFile, engine = 'xlsxwriter')
# grgWB = xlsxwriter.Workbook("GRG Running.xlsx")
grgWB = writer2.book
#worksheet = grgWB.add_worksheet('BGNBD Estimation')

In [5]:
df_input = pd.read_csv("Input.csv")
inputList = df_input["Values"].tolist()
df_input

,Var Names,Values
0,File Name,GABI_BGN Query2 8-8-18.XLSX
1,Start Date Year,2011
2,Start Date Month,1
3,Start Date Day,1
4,End Date Year,2018
5,End Date Month,8
6,End Date Day,6


In [6]:
beginning_date = dt.datetime(int (inputList[1]), int (inputList[2]), int (inputList[3]))
predict_thru = dt.datetime(int (inputList[4]), int (inputList[5]), int (inputList[6]))

In [7]:
def datevalue(date1):
    temp = dt.datetime(1899, 12, 30)    # Note, not 31st Dec but 30th!
    delta = date1 - temp
    return float(delta.days) + (float(delta.seconds) / 86400)

In [8]:
def valuedate(fh):
    fh = int(fh)
    return datetime.fromordinal(datetime(1900, 1, 1).toordinal() + fh - 2)

In [9]:
beginning_date = datevalue(beginning_date)
predict_thru = datevalue(predict_thru)

In [10]:
endDate = beginning_date + .5*(predict_thru-beginning_date)
firstTimeTo = beginning_date + .31 * .5 * (predict_thru-beginning_date)
print(endDate)

41931.0


In [11]:
df_query2 = pd.read_excel(inputList[0]);
df_query2

,Constituent ID,Name,First Gift Date,Last Gift Date,Gift Date,Gift Amount,Gift Type,Inactive?
0,1927780,"Angeles, Joey D.",40547,43158,40547,50.0,Cash,False
1,1927780,"Angeles, Joey D.",40547,43158,40578,50.0,Cash,False
2,1927780,"Angeles, Joey D.",40547,43158,40606,50.0,Cash,False
3,1927780,"Angeles, Joey D.",40547,43158,40637,50.0,Cash,False
4,1927780,"Angeles, Joey D.",40547,43158,40667,50.0,Cash,False
...,...,...,...,...,...,...,...,...
5594,1929858,"Grove, Karen E.",40970,42031,41828,600.0,Cash,False
5595,1929858,"Grove, Karen E.",40970,42031,42031,600.0,Cash,False
5596,1929859,"Leipr, Michael H.",40970,40970,40970,750.0,Cash,True
5597,1929860,"Foy, Charles B.",40970,40970,40970,1200.0,Cash,True


In [12]:
dfraw1 = df_query2.copy(deep=True)
removePledge0 = []
for index, rows in dfraw1.iterrows():
    if(dfraw1.iloc[index, 5] == 0 or dfraw1.iloc[index, 6] == ("Pledge")):
        removePledge0.append(True)
    else:
        removePledge0.append("")
dfraw1['remove $0 & pledges'] = (removePledge0)
dfraw1

,Constituent ID,Name,First Gift Date,Last Gift Date,Gift Date,Gift Amount,Gift Type,Inactive?,remove $0 & pledges
0,1927780,"Angeles, Joey D.",40547,43158,40547,50.0,Cash,False,
1,1927780,"Angeles, Joey D.",40547,43158,40578,50.0,Cash,False,
2,1927780,"Angeles, Joey D.",40547,43158,40606,50.0,Cash,False,
3,1927780,"Angeles, Joey D.",40547,43158,40637,50.0,Cash,False,
4,1927780,"Angeles, Joey D.",40547,43158,40667,50.0,Cash,False,
...,...,...,...,...,...,...,...,...,...
5594,1929858,"Grove, Karen E.",40970,42031,41828,600.0,Cash,False,
5595,1929858,"Grove, Karen E.",40970,42031,42031,600.0,Cash,False,
5596,1929859,"Leipr, Michael H.",40970,40970,40970,750.0,Cash,True,
5597,1929860,"Foy, Charles B.",40970,40970,40970,1200.0,Cash,True,


In [13]:
dfdp1 = dfraw1.copy(deep=True)
pledgeor0 = dfdp1[dfdp1['remove $0 & pledges'] == True].index
dfdp1.drop(pledgeor0, inplace=True)
dfdp1 = dfdp1.reset_index(drop=True)
dfdp1

,Constituent ID,Name,First Gift Date,Last Gift Date,Gift Date,Gift Amount,Gift Type,Inactive?,remove $0 & pledges
0,1927780,"Angeles, Joey D.",40547,43158,40547,50.0,Cash,False,
1,1927780,"Angeles, Joey D.",40547,43158,40578,50.0,Cash,False,
2,1927780,"Angeles, Joey D.",40547,43158,40606,50.0,Cash,False,
3,1927780,"Angeles, Joey D.",40547,43158,40637,50.0,Cash,False,
4,1927780,"Angeles, Joey D.",40547,43158,40667,50.0,Cash,False,
...,...,...,...,...,...,...,...,...,...
5584,1929858,"Grove, Karen E.",40970,42031,41828,600.0,Cash,False,
5585,1929858,"Grove, Karen E.",40970,42031,42031,600.0,Cash,False,
5586,1929859,"Leipr, Michael H.",40970,40970,40970,750.0,Cash,True,
5587,1929860,"Foy, Charles B.",40970,40970,40970,1200.0,Cash,True,


In [14]:
dfraw2 = dfdp1.copy(deep=True)
dfraw2 = dfraw2.drop(columns = ["Gift Amount", "Gift Type", "remove $0 & pledges"])
dfraw2

,Constituent ID,Name,First Gift Date,Last Gift Date,Gift Date,Inactive?
0,1927780,"Angeles, Joey D.",40547,43158,40547,False
1,1927780,"Angeles, Joey D.",40547,43158,40578,False
2,1927780,"Angeles, Joey D.",40547,43158,40606,False
3,1927780,"Angeles, Joey D.",40547,43158,40637,False
4,1927780,"Angeles, Joey D.",40547,43158,40667,False
...,...,...,...,...,...,...
5584,1929858,"Grove, Karen E.",40970,42031,41828,False
5585,1929858,"Grove, Karen E.",40970,42031,42031,False
5586,1929859,"Leipr, Michael H.",40970,40970,40970,True
5587,1929860,"Foy, Charles B.",40970,40970,40970,True


In [15]:
adGiftList = [1]
print(endDate)
for index, row in islice(dfraw2.iterrows(), 1, None):
    if(dfraw2.iloc[index-1, 0] != dfraw2.iloc[index, 0]):
        adGiftList.append(1)
    elif(dfraw2.iloc[index,4]>endDate):
            adGiftList.append(0)
    else:
        if(dfraw2.iloc[index-1, 0] == dfraw2.iloc[index, 0] and dfraw2.iloc[index-1, 4] != dfraw2.iloc[index, 4]):
            adGiftList.append(adGiftList[index-1] + 1)
        else:
            adGiftList.append(adGiftList[index-1])

dfraw2['adj. num gift'] = (adGiftList)

41931.0


In [16]:
keepList = []
for index, row in islice(dfraw2.iterrows(), 1, None):
    if ((dfraw2.iloc[index-1, 3]<=endDate and dfraw2.iloc[index-1,0] != dfraw2.iloc[index,0]) or (dfraw2.iloc[index-1, 3]>endDate and dfraw2.iloc[index-1, 4]<=endDate and dfraw2.iloc[index, 4]>endDate)):
        keepList.append("")
    else:
        keepList.append(False)

        
if(dfraw2.iloc[-1, 3]<=endDate):
    keepList.append("")
else:
    keepList.append(False)
        
dfraw2['KEEP'] = (keepList)

In [17]:
rptList = [""]
for index, row in islice(dfraw2.iterrows(), 1, None):
    if (dfraw2.iloc[index-1, 0] == dfraw2.iloc[index, 0] and dfraw2.iloc[index, 4]>endDate and dfraw2.iloc[index-1, 4] == dfraw2.iloc[index, 4]):
        rptList.append("blah")
    else:
        rptList.append("")
        
dfraw2['rpt gift after cali'] = (rptList)

In [18]:
dfdp2 = dfraw2.drop(columns = 'rpt gift after cali')
falseKeep = dfdp2[dfdp2['KEEP'] == False].index
dfdp2.drop(falseKeep, inplace=True)
dfdp2 = dfdp2.reset_index(drop=True)
dfdp2

,Constituent ID,Name,First Gift Date,Last Gift Date,Gift Date,Inactive?,adj. num gift,KEEP
0,1927780,"Angeles, Joey D.",40547,43158,41718,False,26,
1,1927778,"Ayika, Christopher",40548,40548,40548,True,1,
2,1927784,"Boland, Patrick T.",40550,40613,40613,True,7,
3,1927785,"Callahan, Patrick J.",40554,40554,40554,True,1,
4,1927788,"Rowell, Esperanza",40557,42403,41660,False,4,
...,...,...,...,...,...,...,...,...
2021,1930534,"Chmielarczyk, Allison",40969,40969,40969,True,1,
2022,1929858,"Grove, Karen E.",40970,42031,41828,False,4,
2023,1929859,"Leipr, Michael H.",40970,40970,40970,True,1,
2024,1929860,"Foy, Charles B.",40970,40970,40970,True,1,


In [19]:
dfmd = dfdp2[["Constituent ID", "Name"]]
dfmd["x (#donations)"] = dfdp2["adj. num gift"]-1
dfmd['t_x (last gift)'] = (dfdp2['Gift Date'] - dfdp2['First Gift Date'])/365

<ipython-input-19-ea6a9f9cc4ea>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfmd["x (#donations)"] = dfdp2["adj. num gift"]-1
<ipython-input-19-ea6a9f9cc4ea>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfmd['t_x (last gift)'] = (dfdp2['Gift Date'] - dfdp2['First Gift Date'])/365


In [20]:
tempDate = endDate+1
dfmd['T (total time span)'] = (tempDate - dfdp2['First Gift Date'])/365
dfmd

<ipython-input-20-0793d730c68b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfmd['T (total time span)'] = (tempDate - dfdp2['First Gift Date'])/365


,Constituent ID,Name,x (#donations),t_x (last gift),T (total time span)
0,1927780,"Angeles, Joey D.",25,3.208219,3.794521
1,1927778,"Ayika, Christopher",0,0.000000,3.791781
2,1927784,"Boland, Patrick T.",6,0.172603,3.786301
3,1927785,"Callahan, Patrick J.",0,0.000000,3.775342
4,1927788,"Rowell, Esperanza",3,3.021918,3.767123
...,...,...,...,...,...
2021,1930534,"Chmielarczyk, Allison",0,0.000000,2.638356
2022,1929858,"Grove, Karen E.",3,2.350685,2.635616
2023,1929859,"Leipr, Michael H.",0,0.000000,2.635616
2024,1929860,"Foy, Charles B.",0,0.000000,2.635616


In [21]:
df_BGNBD = dfmd.copy(deep=True)
donationNum = df_BGNBD['x (#donations)'].tolist()
lastGift = df_BGNBD['t_x (last gift)'].tolist()
T = df_BGNBD['T (total time span)'].tolist()

In [22]:
A4 = []
for index, rows in islice(df_BGNBD.iterrows(), 0, None):
    if donationNum[index]>0:
        A4.append(math.log(a)-math.log(b+donationNum[index]-1)-(r+donationNum[index])*math.log(alpha+lastGift[index]))
    else:
        A4.append(0)
df_BGNBD['ln(A_4)'] = (A4)
A_4 = df_BGNBD['ln(A_4)'].tolist()

In [23]:
A3 = []
for index, rows in islice(df_BGNBD.iterrows(), 0, None):
    A3.append(-(r+donationNum[index])*math.log(alpha+T[index]))
df_BGNBD['ln(A_3)'] = (A3)
A_3 = df_BGNBD['ln(A_3)'].tolist()

In [24]:
A2 = []
for index, rows in islice(df_BGNBD.iterrows(), 0, None):
    A2.append(math.lgamma(a+b)+math.lgamma(b+donationNum[index])-math.lgamma(b)-math.lgamma(a+b+donationNum[index]))
df_BGNBD['ln(A_2)'] = (A2)
A_2 = df_BGNBD['ln(A_2)'].tolist()

In [25]:
A1 = []
for index, rows in islice(df_BGNBD.iterrows(), 0, None):
    A1.append(math.lgamma(r+donationNum[index])-math.lgamma(r)+r*math.log(alpha))
df_BGNBD['ln(A_1)'] = (A1)
A_1 = df_BGNBD['ln(A_1)'].tolist()
df_BGNBD

,Constituent ID,Name,x (#donations),t_x (last gift),T (total time span),ln(A_4),ln(A_3),ln(A_2),ln(A_1)
0,1927780,"Angeles, Joey D.",25,3.208219,3.794521,-36.278657,-35.764151,-1.048461,53.258612
1,1927778,"Ayika, Christopher",0,0.000000,3.791781,0.000000,-0.209874,0.000000,-0.154335
2,1927784,"Boland, Patrick T.",6,0.172603,3.786301,1.004611,-8.730776,-0.619533,3.042515
3,1927785,"Callahan, Patrick J.",0,0.000000,3.775342,0.000000,-0.209288,0.000000,-0.154335
4,1927788,"Rowell, Esperanza",3,3.021918,3.767123,-6.156554,-4.455605,-0.437634,-1.165218
...,...,...,...,...,...,...,...,...,...
2021,1930534,"Chmielarczyk, Allison",0,0.000000,2.638356,0.000000,-0.161705,0.000000,-0.154335
2022,1929858,"Grove, Karen E.",3,2.350685,2.635616,-5.458218,-3.444571,-0.437634,-1.165218
2023,1929859,"Leipr, Michael H.",0,0.000000,2.635616,0.000000,-0.161570,0.000000,-0.154335
2024,1929860,"Foy, Charles B.",0,0.000000,2.635616,0.000000,-0.161570,0.000000,-0.154335


In [26]:
ln = []
for index, rows in islice(df_BGNBD.iterrows(), 0, None):
    if donationNum[index]>0:
        ln.append(A_1[index]+A_2[index]+math.log(math.exp(A_3[index])+1*math.exp(A_4[index])))
    else:
        ln.append(A_1[index]+A_2[index]+math.log(math.exp(A_3[index])+0*math.exp(A_4[index])))
df_BGNBD['ln(.)'] = (ln)

In [27]:
BGNBDcols = list(df_BGNBD.columns.values)
df_BGNBD = df_BGNBD[BGNBDcols[0:5] + [BGNBDcols[-1]] + [BGNBDcols[-2]] + [BGNBDcols[-3]] + [BGNBDcols[-4]] + [BGNBDcols[-5]]]
df_BGNBD

,Constituent ID,Name,x (#donations),t_x (last gift),T (total time span),ln(.),ln(A_1),ln(A_2),ln(A_3),ln(A_4)
0,1927780,"Angeles, Joey D.",25,3.208219,3.794521,16.914625,53.258612,-1.048461,-35.764151,-36.278657
1,1927778,"Ayika, Christopher",0,0.000000,3.791781,-0.364210,-0.154335,0.000000,-0.209874,0.000000
2,1927784,"Boland, Patrick T.",6,0.172603,3.786301,3.427652,3.042515,-0.619533,-8.730776,1.004611
3,1927785,"Callahan, Patrick J.",0,0.000000,3.775342,-0.363623,-0.154335,0.000000,-0.209288,0.000000
4,1927788,"Rowell, Esperanza",3,3.021918,3.767123,-5.890817,-1.165218,-0.437634,-4.455605,-6.156554
...,...,...,...,...,...,...,...,...,...,...
2021,1930534,"Chmielarczyk, Allison",0,0.000000,2.638356,-0.316040,-0.154335,0.000000,-0.161705,0.000000
2022,1929858,"Grove, Karen E.",3,2.350685,2.635616,-4.922112,-1.165218,-0.437634,-3.444571,-5.458218
2023,1929859,"Leipr, Michael H.",0,0.000000,2.635616,-0.315905,-0.154335,0.000000,-0.161570,0.000000
2024,1929860,"Foy, Charles B.",0,0.000000,2.635616,-0.315905,-0.154335,0.000000,-0.161570,0.000000


In [28]:
dfGRG = pd.DataFrame()
dfGRG["col1"] = ["r","alpha","a","b", "LL"]
dfGRG["col2"] = [1,1,1,1,sum(df_BGNBD["ln(.)"])]
dfGRG

,col1,col2
0,r,1.000000
1,alpha,1.000000
2,a,1.000000
3,b,1.000000
4,LL,-2303.317031


In [29]:
dfGRG.to_excel(writer2, "BGNBD Estimation", index = False, header = False, startrow=0, startcol=0)
df_BGNBD.to_excel(writer2, "BGNBD Estimation", index = False, startrow = 6, startcol=0)
worksheet = grgWB.get_worksheet_by_name('BGNBD Estimation')
worksheet.write_formula('B5', '=SUM(F:F)')
worksheet.write_array_formula(7, 5, 7+len(ln)-1, 5, '=G8:G{}+H8:H{}+LN(EXP(I8:I{})+(C8:C{}>0)*EXP(J8:J{}))'.format(7+len(ln),7+len(ln),7+len(ln),7+len(ln),7+len(ln)))
worksheet.write_array_formula(7, 6, 7+len(ln)-1, 6, '=GAMMALN(B$1+C8:C{})-GAMMALN(B$1)+B$1*LN(B$2)'.format(7+len(ln)))
worksheet.write_array_formula(7, 7, 7+len(ln)-1, 7, '=GAMMALN(B$3+B$4)+GAMMALN(B$4+C8:C{})-GAMMALN(B$4)-GAMMALN(B$3+B$4+C8:C{})'.format(7+len(ln),7+len(ln)))
worksheet.write_array_formula(7, 8, 7+len(ln)-1, 8, '=-(B$1+C8:C{})*LN(B$2+E8:E{})'.format(7+len(ln),7+len(ln)))
worksheet.write_array_formula(7, 9, 7+len(ln)-1, 9, '=IF(C8:C{}>0, LN(B$3)-LN(B$4+C8:C{}-1)-(B$1+C8:C{})*LN(B$2+D8:D{}),0)'.format(7+len(ln),7+len(ln),7+len(ln),7+len(ln)))
#writer2.save()
grgWB.close()

In [30]:
dfraw1.to_excel(writer, 'Raw data1', index = False)
dfdp1.to_excel(writer, 'Data Prep1', index = False)
dfraw2.to_excel(writer, 'Raw data2', index = False)
dfdp2.to_excel(writer, 'Data Prep2', index = False)
dfmd.to_excel(writer, 'Model Data', index = False)
df_BGNBD.to_excel(writer, "BGNBD Estimation", index = False)
writer.save()